In [33]:
import pandas as pd

df = pd.read_csv('C:/Users/Subir/Downloads/pnlp_study_project-master/notebooks/pnlp_data_en.csv', delimiter=';')
df.dropna()

# rename columns for easier usability
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
# preview
df.head(10)

,report_grouping,question_text,comments
0,Large Department,Please tell us what is working well.,"we do what our customers need, we communicate ..."
1,Large Department,Please tell us what is working well.,Customs business development continues to grow...
2,Large Department,Please tell us what is working well.,"I think the team work hard, are committed to c..."
3,Large Department,Please tell us what is working well.,Overall working towards a customer centric env...
4,Large Department,Please tell us what is working well.,Customer centricity is a growing culture in th...
5,Large Department,Please tell us what is working well.,•Develop a comfortable rapport with clients an...
6,Large Department,Please tell us what is working well.,THE CUSTOMER IS THE CENTER.
7,Large Department,Please tell us what is working well.,"we are usually seeking customer satisfaction, ..."
8,Large Department,Please tell us what is working well.,Alignment between regional office and country ...
9,Large Department,Please tell us what is working well.,"innovation, customer relations ship and custom..."


In [34]:
len(df)

17672

In [35]:
#!pip install pyLDAvis

In [36]:
from bs4 import BeautifulSoup
import re, nltk, string
import numpy as np
import pandas as pd
from pprint import pprint
from html.parser import HTMLParser

# Gensim
import gensim
#import gensim.corpora as corpora
#from gensim.utils import simple_preprocess
#from gensim.models import CoherenceModel

In [37]:
# Apostrophe Dictionary and if you have more words in mind, please add it in the bottom
apostrophe = {
    "aren't" : "are not",
    "can't" : "cannot",
    "couldn't" : "could not",
    "didn't" : "did not",
    "doesn't" : "does not",
    "don't" : "do not",
    "hadn't" : "had not",
    "hasn't" : "has not",
    "haven't" : "have not",
    "he'd" : "he would",
    "he'll" : "he will",
    "he's" : "he is",
    "i'd" : "I would",
    "i'd" : "I had",
    "i'll" : "I will",
    "i'm" : "I am",
    "isn't" : "is not",
    "it's" : "it is",
    "it'll":"it will",
    "i've" : "I have",
    "let's" : "let us",
    "mightn't" : "might not",
    "mustn't" : "must not",
    "shan't" : "shall not",
    "she'd" : "she would",
    "she'll" : "she will",
    "she's" : "she is",
    "shouldn't" : "should not",
    "that's" : "that is",
    "there's" : "there is",
    "they'd" : "they would",
    "they'll" : "they will",
    "they're" : "they are",
    "they've" : "they have",
    "we'd" : "we would",
    "we're" : "we are",
    "weren't" : "were not",
    "we've" : "we have",
    "what'll" : "what will",
    "what're" : "what are",
    "what's" : "what is",
    "what've" : "what have",
    "where's" : "where is",
    "who'd" : "who would",
    "who'll" : "who will",
    "who're" : "who are",
    "who's" : "who is",
    "who've" : "who have",
    "won't" : "will not",
    "wouldn't" : "would not",
    "you'd" : "you would",
    "you'll" : "you will",
    "you're" : "you are",
    "you've" : "you have",
    "'re": " are",
    "wasn't": "was not",
    "we'll":" will",
    "didn't": "did not",
    "'s": "is",
    "'re": "are"
}

#Short words dictionary and if have more words in mind, please add it in the bottom
short_words = {
"121": "one to one",
"a/s/l": "age, sex, location",
"adn": "any day now",
"afaik": "as far as I know",
"afk": "away from keyboard",
"aight": "alright",
"alol": "actually laughing out loud",
"b4": "before",
"b4n": "bye for now",
"bak": "back at the keyboard",
"bf": "boyfriend",
"bff": "best friends forever",
"bfn": "bye for now",
"bg": "big grin",
"bta": "but then again",
"btw": "by the way",
"cid": "crying in disgrace",
"cnp": "continued in my next post",
"cp": "chat post",
"cu": "see you",
"cul": "see you later",
"cul8r": "see you later",
"cya": "bye",
"cyo": "see you online",
"dbau": "doing business as usual",
"fud": "fear, uncertainty, and doubt",
"fwiw": "for what it's worth",
"fyi": "for your information",
"g": "grin",
"g2g": "got to go",
"ga": "go ahead",
"gal": "get a life",
"gf": "girlfriend",
"gfn": "gone for now",
"gmbo": "giggling my butt off",
"gmta": "great minds think alike",
"h8": "hate",
"hagn": "have a good night",
"hdop": "help delete online predators",
"hhis": "hanging head in shame",
"iac": "in any case",
"ianal": "I am not a lawyer",
"ic": "I see",
"idk": "I don't know",
"imao": "in my arrogant opinion",
"imnsho": "in my not so humble opinion",
"imo": "in my opinion",
"iow": "in other words",
"ipn": "I’m posting naked",
"irl": "in real life",
"jk": "just kidding",
"l8r": "later",
"ld": "later, dude",
"ldr": "long distance relationship",
"llta": "lots and lots of thunderous applause",
"lmao": "laugh my ass off",
"lmirl": "let's meet in real life",
"lol": "laugh out loud",
"ltr": "longterm relationship",
"lulab": "love you like a brother",
"lulas": "love you like a sister",
"luv": "love",
"m/f": "male or female",
"m8": "mate",
"milf": "mother I would like to fuck",
"oll": "online love",
"omg": "oh my god",
"otoh": "on the other hand",
"pir": "parent in room",
"ppl": "people",
"r": "are",
"rofl": "roll on the floor laughing",
"rpg": "role playing games",
"ru": "are you",
"shid": "slaps head in disgust",
"somy": "sick of me yet",
"sot": "short of time",
"thanx": "thanks",
"thx": "thanks",
"ttyl": "talk to you later",
"u": "you",
"ur": "you are",
"uw": "you’re welcome",
"wb": "welcome back",
"wfm": "works for me",
"wibni": "wouldn't it be nice if",
"wtf": "what the fuck",
"wtg": "way to go",
"wtgp": "want to go private",
"ym": "young man",
"gr8": "great"
}

def apos_short_dict(text, dictionary):
    for word in text.split():
        if word.lower() in dictionary:
            if word.lower() in text.split():
                text = text.replace(word, dictionary[word.lower()])
    return text

df.head(5)

,report_grouping,question_text,comments
0,Large Department,Please tell us what is working well.,"we do what our customers need, we communicate ..."
1,Large Department,Please tell us what is working well.,Customs business development continues to grow...
2,Large Department,Please tell us what is working well.,"I think the team work hard, are committed to c..."
3,Large Department,Please tell us what is working well.,Overall working towards a customer centric env...
4,Large Department,Please tell us what is working well.,Customer centricity is a growing culture in th...


In [38]:
def remove_html(txt):
    '''Remove HTML'''
    txt = BeautifulSoup(txt, 'lxml')
    return txt.get_text()

def remove_punctuation(text):
    '''Remove any punctuation'''
    return "".join([i for i in text if i not in string.punctuation])

def remove_stopwords(text):
    '''Remove stop words'''
    return "".join([w for w in text if w not in stopwords.words('english')])   

'''
    Control the parameter by putting value of TRUE or FALSE according to requirements. 
    Args : txt - Provided text for preprocessing
            punctuation - Remove all punctuation, Initially value = False
            stopwords - Remove such words which does not have much meaning to a line of text
            correct_apos - Remove apostrophe
            shortwords - Convert any short word to full meaningfull word
            specialCharacter - Replace all specialCharacter
            numbers - Remove numbers
            singleChar - Removing words whom length is one
          
'''
def preprocessing(txt, punctuation= False, stopwords= False, correct_apos= False, 
                  shortWords= False, specialCharacter= False, numbers= False, singleChar= False):
    
    cleanedTxt = txt.apply(lambda x: remove_html(x))
    
    if punctuation:
        cleanedTxt = cleanedTxt.apply(lambda x:remove_punctuation(x))
        
    if stopwords:
        cleanedTxt = cleanedTxt.apply(lambda x: remove_stopwords(x))
        
    if correct_apos:
        cleanedTxt = cleanedTxt.apply(lambda x: apos_short_dict(str(x),apostrophe))
        
    if shortWords:
        cleanedTxt = cleanedTxt.apply(lambda x: apos_short_dict(str(x),short_words))
    
    if specialCharacter:
        '''Replacing Special Characters with space'''
        cleanedTxt = cleanedTxt.apply(lambda x: re.sub(r'[^a-zA-Z0-9]',' ',str(x)))
    
    if numbers:
        '''Replacing Numbers with space'''
        cleanedTxt = cleanedTxt.apply(lambda x: re.sub(r'[^a-zA-Z]',' ',x))
        
    if singleChar:
        '''Removing words whom length is one'''
        cleanedTxt = cleanedTxt.apply(lambda x: ' '.join([w for w in x.split() if len(w)>1]))
    
    return cleanedTxt
      
df['comments'] = preprocessing(df['comments'], punctuation= True, stopwords= False, 
                                correct_apos= True, shortWords= True, specialCharacter= True, 
                               numbers= True, singleChar= True)
df.head(5)

,report_grouping,question_text,comments
0,Large Department,Please tell us what is working well.,we do what our customers need we communicate a...
1,Large Department,Please tell us what is working well.,Customs business development continues to grow...
2,Large Department,Please tell us what is working well.,think the team work hard are committed to cont...
3,Large Department,Please tell us what is working well.,Overall working towards customer centric envir...
4,Large Department,Please tell us what is working well.,Customer centricity is growing culture in the ...


In [39]:
data_words = np.array(df['comments'])
data_words

array(['we do what our customers need we communicate aperiodically',
       'Customs business development continues to grow and expand through the use of our internal business network the team of people which are in place at the moment',
       'think the team work hard are committed to continuous improvement and doing good job for our customers',
       ..., 'not enough space provided',
       'Name have no respect for due to his attitudework ethic and things have seen and heard from him about me and otherskeeps me wondering if this is the right place for driver like myself',
       'Other'], dtype=object)

In [40]:
#Create Biagram & Trigram Models 
from gensim.models import Phrases
# Add bigrams and trigrams
bigram = Phrases(data_words, min_count=10)
trigram = Phrases(bigram[data_words])

for idx in range(len(data_words)):
    for token in bigram[data_words[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            data_words[idx].append(token)
    for token in trigram[data_words[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            data_words[idx].append(token)

In [41]:
# Lemmatize with POS Tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

# 1. Init Lemmatizer
lemmatizer = WordNetLemmatizer()
df['trigram_words'] = data_words
nltk.download('punkt')

def Lemmatize (text):
      return [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)]
df['trigram_words'] = df['trigram_words'].apply(lambda x: Lemmatize(x))
df.head(5)


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Subir\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Subir\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Subir\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,report_grouping,question_text,comments,trigram_words
0,Large Department,Please tell us what is working well.,we do what our customers need we communicate a...,"[we, do, what, our, customer, need, we, commun..."
1,Large Department,Please tell us what is working well.,Customs business development continues to grow...,"[Customs, business, development, continue, to,..."
2,Large Department,Please tell us what is working well.,think the team work hard are committed to cont...,"[think, the, team, work, hard, be, commit, to,..."
3,Large Department,Please tell us what is working well.,Overall working towards customer centric envir...,"[Overall, work, towards, customer, centric, en..."
4,Large Department,Please tell us what is working well.,Customer centricity is growing culture in the ...,"[Customer, centricity, be, grow, culture, in, ..."


In [42]:
# Create Dictionary
id2word = gensim.corpora.Dictionary(array(df['trigram_words']))

# Create Corpus
texts = array(df['trigram_words'])

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 2), (7, 1)]]


In [55]:
# Set parameters for LDA model
num_topics = 10
chunksize = 100 
passes = 10 
iterations = 400
update_every = 1
random_state = 100

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=num_topics,
                                            random_state=random_state,
                                            update_every=update_every,
                                            chunksize=chunksize,
                                            alpha='auto', 
                                            eta='auto',
                                            iterations=iterations,
                                            passes=passes)

In [56]:
# Print the Keyword in the 5 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.065*"staff" + 0.048*"pay" + 0.046*"salary" + 0.040*"provide" + '
  '0.032*"issue" + 0.030*"come" + 0.029*"how" + 0.029*"learn" + 0.026*"hire" + '
  '0.020*"opportunity"'),
 (1,
  '0.058*"each" + 0.042*"less" + 0.038*"ALL" + 0.035*"without" + 0.034*"far" + '
  '0.033*"start" + 0.032*"strategy" + 0.027*"Company" + 0.025*"cant" + '
  '0.022*"treat"'),
 (2,
  '0.058*"THE" + 0.054*"TO" + 0.035*"AND" + 0.029*"WE" + 0.024*"IS" + '
  '0.020*"IN" + 0.018*"MORE" + 0.016*"IT" + 0.015*"OUR" + 0.015*"NEED"'),
 (3,
  '0.067*"than" + 0.054*"high" + 0.038*"off" + 0.036*"now" + 0.035*"into" + '
  '0.034*"Salary" + 0.029*"such" + 0.027*"group" + 0.025*"night" + '
  '0.022*"faster"'),
 (4,
  '0.095*"so" + 0.050*"them" + 0.045*"More" + 0.038*"different" + '
  '0.037*"order" + 0.035*"responsibility" + 0.034*"My" + 0.029*"program" + '
  '0.029*"member" + 0.024*"his"'),
 (5,
  '0.139*"and" + 0.055*"work" + 0.039*"our" + 0.036*"on" + 0.035*"with" + '
  '0.027*"improve" + 0.025*"should" + 0.025*"well

In [57]:
from gensim.models import CoherenceModel

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=array(df['trigram_words']), dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.4050187381444646


In [58]:
import os
from gensim.models.wrappers import LdaMallet

os.environ['MALLET_HOME'] = 'C:\\mallet'

mallet_path = 'C:\\mallet\\bin\\mallet.bat'
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=10, id2word=id2word)

In [59]:
# Show Topics
pprint(ldamallet.print_topics())
doc_ldamallet = ldamallet[corpus]

[(0,
  '0.147*"team" + 0.057*"department" + 0.048*"support" + 0.045*"improvement" + '
  '0.039*"great" + 0.037*"work" + 0.028*"Team" + 0.023*"teamwork" + '
  '0.018*"goal" + 0.017*"achieve"'),
 (1,
  '0.069*"job" + 0.056*"people" + 0.032*"manager" + 0.023*"supervisor" + '
  '0.023*"thing" + 0.023*"dont" + 0.020*"result" + 0.020*"respect" + '
  '0.017*"direct" + 0.014*"feedback"'),
 (2,
  '0.125*"customer" + 0.060*"service" + 0.039*"business" + 0.033*"improve" + '
  '0.030*"focus" + 0.029*"level" + 0.027*"quality" + 0.021*"station" + '
  '0.016*"happy" + 0.014*"operation"'),
 (3,
  '0.070*"time" + 0.035*"system" + 0.016*"require" + 0.015*"information" + '
  '0.015*"market" + 0.014*"order" + 0.012*"tool" + 0.012*"current" + '
  '0.012*"term" + 0.011*"long"'),
 (4,
  '0.067*"staff" + 0.037*"day" + 0.037*"year" + 0.029*"salary" + '
  '0.027*"increase" + 0.020*"high" + 0.017*"provide" + 0.014*"cost" + '
  '0.013*"pay" + 0.012*"facility"'),
 (5,
  '0.119*"company" + 0.025*"WORK" + 0.025*"TEA

In [60]:
# Compute Coherence Score ldamallet
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=array(df['trigram_words']), dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)


Coherence Score:  0.4232139893291868


In [63]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model=gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=10, id2word=dictionary)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [64]:
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=array(df['trigram_words']), start=0, limit=10, step=1)

limit=10; start=0; step=1;
x = range(start, limit, step)

# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

Num Topics = 0  has Coherence Value of 0.4289
Num Topics = 1  has Coherence Value of 0.4016
Num Topics = 2  has Coherence Value of 0.3874
Num Topics = 3  has Coherence Value of 0.4351
Num Topics = 4  has Coherence Value of 0.4023
Num Topics = 5  has Coherence Value of 0.4116
Num Topics = 6  has Coherence Value of 0.3966
Num Topics = 7  has Coherence Value of 0.4088
Num Topics = 8  has Coherence Value of 0.4332
Num Topics = 9  has Coherence Value of 0.4209


In [65]:
# Select the model and print the topics
# Here best model corerence value is 0.4351
optimal_model = model_list[3]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

[(0,
  '0.125*"customer" + 0.060*"service" + 0.039*"business" + 0.030*"focus" + '
  '0.027*"quality" + 0.023*"level" + 0.021*"improve" + 0.019*"provide" + '
  '0.016*"product" + 0.014*"internal"'),
 (1,
  '0.064*"time" + 0.064*"staff" + 0.031*"provide" + 0.029*"office" + '
  '0.025*"issue" + 0.023*"area" + 0.017*"back" + 0.016*"require" + '
  '0.013*"facility" + 0.011*"long"'),
 (2,
  '0.061*"job" + 0.055*"people" + 0.043*"make" + 0.035*"employee" + '
  '0.034*"thing" + 0.032*"manager" + 0.024*"feel" + 0.023*"supervisor" + '
  '0.022*"dont" + 0.022*"understand"'),
 (3,
  '0.114*"employee" + 0.073*"training" + 0.043*"improvement" + 0.037*"give" + '
  '0.034*"development" + 0.034*"learn" + 0.027*"opportunity" + '
  '0.022*"activity" + 0.020*"knowledge" + 0.020*"result"'),
 (4,
  '0.210*"team" + 0.170*"work" + 0.047*"support" + 0.028*"Team" + '
  '0.022*"member" + 0.019*"leader" + 0.018*"goal" + 0.017*"experience" + '
  '0.017*"achieve" + 0.015*"task"'),
 (5,
  '0.116*"company" + 0.060*"m

In [67]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=df['trigram_words']):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Row_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'comments']

# Show
df_dominant_topic.head(20)

,Row_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,comments
0,0,0.0,0.1321,"customer, service, business, focus, quality, l...","[we, do, what, our, customer, need, we, commun..."
1,1,0.0,0.1517,"customer, service, business, focus, quality, l...","[Customs, business, development, continue, to,..."
2,2,4.0,0.1488,"team, work, support, Team, member, leader, goa...","[think, the, team, work, hard, be, commit, to,..."
3,3,6.0,0.1303,"work, company, good, great, environment, lot, ...","[Overall, work, towards, customer, centric, en..."
4,4,0.0,0.1318,"customer, service, business, focus, quality, l...","[Customer, centricity, be, grow, culture, in, ..."
5,5,0.0,0.1517,"customer, service, business, focus, quality, l...","[Develop, comfortable, rapport, with, client, ..."
6,6,8.0,0.1346,"department, teamwork, WORK, TEAM, Communicatio...","[THE, CUSTOMER, IS, THE, CENTER]"
7,7,0.0,0.1552,"customer, service, business, focus, quality, l...","[we, be, usually, seek, customer, satisfaction..."
8,8,0.0,0.1833,"customer, service, business, focus, quality, l...","[Alignment, between, regional, office, and, co..."
9,9,0.0,0.1429,"customer, service, business, focus, quality, l...","[innovation, customer, relation, ship, and, cu..."


In [ ]:
## TODO
## Clean data to increase the performance of model
## Try other model like LSA, PLSA
## Divide the dataset accroding to two question and assign topic
## Imporove the permormance using hypertuning if possible or check the coherence score using U-mass instead of c_v
## then use LSTM model to classify text based on topic